In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np

In [115]:
path = r'D:\NUS\project\LTA_data\od_comuting_flow\bus\2021'

szone = gpd.read_file(r"D:\NUS\project\Singapore_subzone\subzone_2019\MasterPlan_SUBZONE_2019\master-plan-2019-subzone-boundary-no-sea-geojson-polygon.shp")
lis_szone = list(set(szone['SUBZONE_N']))
print('the number of subzone is: {}'.format(len(lis_szone)))

the number of subzone is: 332


In [133]:
lis_file = os.listdir(path=path)
lis_file   

['bus_commuting_flow_202101.csv',
 'bus_commuting_flow_202102.csv',
 'bus_commuting_flow_202103.csv',
 'bus_commuting_flow_202104.csv',
 'bus_commuting_flow_202105.csv',
 'bus_commuting_flow_202106.csv',
 'bus_commuting_flow_202107.csv',
 'bus_commuting_flow_202108.csv',
 'bus_commuting_flow_202109.csv',
 'bus_commuting_flow_202110.csv',
 'bus_commuting_flow_202111.csv',
 'bus_commuting_flow_202112.csv']

In [120]:
def get_raw_data(path,i):
    lis_file = os.listdir(path=path)
    
    df = pd.read_csv(path+'\\'+lis_file[i])
    
    return df


def aggregate_od(od):
    origin,destination,total_trip = 'Origin_Subzone','Destination_Subzone','TOTAL_TRIPS'
    df = od.groupby([origin,destination]).sum(total_trip).reset_index()
    return df


def creat_szone_matrix(lis):
    cols = ['SUBZONE_N']
    for i in lis:
        cols.append(i)
    szone_matrix = pd.DataFrame(columns = cols)
    szone_matrix['SUBZONE_N'] = lis
    
    return szone_matrix


def output_destination(subzone,df):
    dataframe = df.loc[df['Origin_Subzone'] == subzone].copy()
    dataframe.index = range(len(dataframe))
    lis_destination = dataframe.loc[:,'Destination_Subzone']
    lis_total_trip = dataframe.loc[:,'TOTAL_TRIPS']
    
    return list(lis_destination),list(lis_total_trip)


def commuting_flow_matching(szone_m,subzone,lis_sz,lis_trip):
    index = szone_m.loc[szone_m['SUBZONE_N'] == subzone].index
    for i in range(len(lis_szone)):
        for x in range(len(lis_sz)):
            if lis_sz[x] == lis_szone[i]:
                szone_m.loc[index,lis_szone[i]] = lis_trip[x]               
    return szone_m

In [143]:
def main():
    path = r'D:\NUS\project\LTA_data\od_comuting_flow\bus\2021'
    szone = gpd.read_file(r"D:\NUS\project\Singapore_subzone\subzone_2019\MasterPlan_SUBZONE_2019\master-plan-2019-subzone-boundary-no-sea-geojson-polygon.shp")
    lis_szone = list(set(szone['SUBZONE_N']))
    lis_szone.sort()
    
    lis_file = os.listdir(path=path)
    print(lis_file[3])  
    
    commuting_flow = get_raw_data(path,3)
    aggregate_commuting_flow = aggregate_od(commuting_flow)
    szone_matrix = creat_szone_matrix(lis_szone)
    
    for i in lis_szone:
        lis_d,lis_trip = output_destination(i,aggregate_commuting_flow)
        szone_matrix = commuting_flow_matching(szone_matrix,i,lis_d,lis_trip)
        
    szone_matrix = szone_matrix.fillna(0)
    return szone_matrix

In [144]:
if __name__ == '__main__':
    bus_od_subzone_matrix = main()

bus_commuting_flow_202104.csv


In [145]:
result_path = r'D:\NUS\project\LTA_data\commuting_flow_matrix\bus'

In [146]:
bus_od_subzone_matrix.to_csv(result_path+'\\'+'bus_commuting_flow_matrix_202104_by_subzone.csv',index = False)

In [147]:
bus_od_subzone_matrix

,SUBZONE_N,ADMIRALTY,AIRPORT ROAD,ALEXANDRA HILL,ALEXANDRA NORTH,ALJUNIED,ANAK BUKIT,ANCHORVALE,ANG MO KIO TOWN CENTRE,ANSON,...,YIO CHU KANG EAST,YIO CHU KANG NORTH,YIO CHU KANG WEST,YISHUN CENTRAL,YISHUN EAST,YISHUN SOUTH,YISHUN WEST,YUHUA EAST,YUHUA WEST,YUNNAN
0,ADMIRALTY,13670,0,0,0,0,0,0,0,0,...,0,0,0,15703,166,2026,25703,0,0,0
1,AIRPORT ROAD,0,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALEXANDRA HILL,0,0,32339,1307,1562,1440,0,0,983,...,0,0,0,1074,0,0,226,41,1080,0
3,ALEXANDRA NORTH,0,0,777,1886,396,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALJUNIED,0,0,1553,0,259360,438,955,1762,1047,...,300,0,876,144,0,0,0,25,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,YISHUN SOUTH,0,0,1381,0,67,400,6294,9162,0,...,10,16754,3229,144204,68394,90254,45108,0,0,0
328,YISHUN WEST,22700,0,0,0,0,0,0,0,0,...,0,0,0,131251,58934,57100,305593,0,0,0
329,YUHUA EAST,0,0,620,0,0,1534,0,0,0,...,0,0,0,0,0,0,0,22955,48863,3753
330,YUHUA WEST,0,0,1186,0,0,4122,0,0,0,...,0,0,0,0,0,0,0,44449,41798,12554
